<a href="https://colab.research.google.com/github/cuongdaoo/Normalization-of-Property-Data-Using-Edit-Distance-and-Word-Embedding./blob/main/INTERNSHIP_PROJECT_ON_STANDARDIZING_HOUSE_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORTING ESSENTIAL LIBRABIES

In [1]:
!pip install emoji
!pip install pandasql
!pip install sentence-transformers
!pip install nltk spacy numpy
!python -m spacy download en_core_web_sm
!pip install git+https://github.com/boudinfl/pke.git
!pip install unidecode


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 38.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-3m68pda_
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-3m68pda_
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done


In [2]:
import emoji
import string
import numpy as np
import pandas as pd
import pandasql as ps
import unidecode

# Natural Language Processing packages
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import re
import nltk
from nltk.metrics import edit_distance
import torch
import torch.nn.functional as F

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Normalize MinMaxScaler function to the range [0,1]
def minmax_scaler(x):
    return (x - x.min()) / (x.max() - x.min()) if x.max() > x.min() else 0

#IMPORTING DATA

In [4]:
data_street_training=pd.read_csv('/content/ten_duong.csv')
data_address=pd.read_csv('/content/dia_chinh_v3.csv')

In [5]:
data_street_training

,Street_ID,Street_Name
0,1,Đặng Văn Ngữ
1,2,Nguyễn Văn Cừ
2,3,Trần Hưng Đạo
3,4,Phạm Văn Đồng
4,5,Nguyễn Tri Phương
...,...,...
95,96,Phan Đăng Lưu
96,97,Huỳnh Văn Nghệ
97,98,Trương Quốc Dung
98,99,Nguyễn Hữu Thọ


In [6]:
data_address

,Street_ID,Street_Name,Error_Type
0,1,"SN 19C, Đặng Văn Ngữ",không lỗi
1,1,"SN 364, Đặng ăVn Ngữ",lỗi đổi thứ tự các từ
2,1,"SN 41B, Ñaëng Vaên Ngöõ",lỗi font
3,1,"SN 526C, Đặng Văn Ngữ",lỗi thiếu từ + đổi thứ tự các từ
4,1,"SN 601, Đặng Văn",lỗi sai chính tả + thiếu từ
...,...,...,...
1095,100,"SN 723C, Ñaëng Vaên Saâm",lỗi font
1096,100,"SN 810/57, Nhà trống Đặng Văn Sâm",lỗi thêm từ gây nhiễu
1097,100,"SN 832, Đặgn Văn Sâm",lỗi đổi thứ tự các từ
1098,100,"SN 939, Đặng Văn Sẩm",lỗi sai chính tả


#CLEANING DATA

In [7]:
#Copy raw data
data_address['Street_Name_Raw']=data_address['Street_Name']
data_street_training['Street_Name_Raw']=data_street_training['Street_Name']

In [8]:

def clean(text):
    # Convert a string to uppercase
    text = str(text).upper()
    # Remove characters within square bracket
    text = re.sub('\[.*?\]', '', text)
    # Remove HTML tags
    text = re.sub('https?://\S+|www\.\S+', '', text)
    # Remove web URLs
    text = re.sub('<.*?>+', '', text)
    # Remove punctuation marks
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # Remove newline characters
    text = re.sub('\n', '', text)
    # Remove_vietnamese_accents
    text= unidecode.unidecode(text)

    # Remove emoji
    text = emoji.replace_emoji(text)
    return text

In [9]:
data_address['Street_Name']=data_address['Street_Name'].apply(lambda x:clean(x))
data_street_training['Street_Name']=data_street_training['Street_Name'].apply(lambda x:clean(x))

In [10]:
data_address

,Street_ID,Street_Name,Error_Type,Street_Name_Raw
0,1,SN 19C DANG VAN NGU,không lỗi,"SN 19C, Đặng Văn Ngữ"
1,1,SN 364 DANG AVN NGU,lỗi đổi thứ tự các từ,"SN 364, Đặng ăVn Ngữ"
2,1,SN 41B NAENG VAEN NGOO,lỗi font,"SN 41B, Ñaëng Vaên Ngöõ"
3,1,SN 526C DANG VAN NGU,lỗi thiếu từ + đổi thứ tự các từ,"SN 526C, Đặng Văn Ngữ"
4,1,SN 601 DANG VAN,lỗi sai chính tả + thiếu từ,"SN 601, Đặng Văn"
...,...,...,...,...
1095,100,SN 723C NAENG VAEN SAAM,lỗi font,"SN 723C, Ñaëng Vaên Saâm"
1096,100,SN 81057 NHA TRONG DANG VAN SAM,lỗi thêm từ gây nhiễu,"SN 810/57, Nhà trống Đặng Văn Sâm"
1097,100,SN 832 DAGN VAN SAM,lỗi đổi thứ tự các từ,"SN 832, Đặgn Văn Sâm"
1098,100,SN 939 DANG VAN SAM,lỗi sai chính tả,"SN 939, Đặng Văn Sẩm"


In [11]:
data_street_training

,Street_ID,Street_Name,Street_Name_Raw
0,1,DANG VAN NGU,Đặng Văn Ngữ
1,2,NGUYEN VAN CU,Nguyễn Văn Cừ
2,3,TRAN HUNG DAO,Trần Hưng Đạo
3,4,PHAM VAN DONG,Phạm Văn Đồng
4,5,NGUYEN TRI PHUONG,Nguyễn Tri Phương
...,...,...,...
95,96,PHAN DANG LUU,Phan Đăng Lưu
96,97,HUYNH VAN NGHE,Huỳnh Văn Nghệ
97,98,TRUONG QUOC DUNG,Trương Quốc Dung
98,99,NGUYEN HUU THO,Nguyễn Hữu Thọ


In [12]:
data_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Street_ID        1100 non-null   int64 
 1   Street_Name      1100 non-null   object
 2   Error_Type       1100 non-null   object
 3   Street_Name_Raw  1100 non-null   object
dtypes: int64(1), object(3)
memory usage: 34.5+ KB


In [13]:
data_street_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Street_ID        100 non-null    int64 
 1   Street_Name      100 non-null    object
 2   Street_Name_Raw  100 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


#EDIT DISTANCE


In [14]:
e_data_training=data_street_training.copy()
e_data_address=data_address.copy()

In [15]:
e_data_training['Street_Name'] = e_data_training['Street_Name'].str.replace(r'\s+', '', regex=True)
e_data_training['Street_Name'] = e_data_training['Street_Name'].str.upper()
e_data_address['Street_Name'] = e_data_address['Street_Name'].str.replace(r'\s+', '', regex=True)
e_data_address['Street_Name'] = e_data_address['Street_Name'].str.upper()

In [16]:
e_data_training

,Street_ID,Street_Name,Street_Name_Raw
0,1,DANGVANNGU,Đặng Văn Ngữ
1,2,NGUYENVANCU,Nguyễn Văn Cừ
2,3,TRANHUNGDAO,Trần Hưng Đạo
3,4,PHAMVANDONG,Phạm Văn Đồng
4,5,NGUYENTRIPHUONG,Nguyễn Tri Phương
...,...,...,...
95,96,PHANDANGLUU,Phan Đăng Lưu
96,97,HUYNHVANNGHE,Huỳnh Văn Nghệ
97,98,TRUONGQUOCDUNG,Trương Quốc Dung
98,99,NGUYENHUUTHO,Nguyễn Hữu Thọ


In [17]:
e_data_address

,Street_ID,Street_Name,Error_Type,Street_Name_Raw
0,1,SN19CDANGVANNGU,không lỗi,"SN 19C, Đặng Văn Ngữ"
1,1,SN364DANGAVNNGU,lỗi đổi thứ tự các từ,"SN 364, Đặng ăVn Ngữ"
2,1,SN41BNAENGVAENNGOO,lỗi font,"SN 41B, Ñaëng Vaên Ngöõ"
3,1,SN526CDANGVANNGU,lỗi thiếu từ + đổi thứ tự các từ,"SN 526C, Đặng Văn Ngữ"
4,1,SN601DANGVAN,lỗi sai chính tả + thiếu từ,"SN 601, Đặng Văn"
...,...,...,...,...
1095,100,SN723CNAENGVAENSAAM,lỗi font,"SN 723C, Ñaëng Vaên Saâm"
1096,100,SN81057NHATRONGDANGVANSAM,lỗi thêm từ gây nhiễu,"SN 810/57, Nhà trống Đặng Văn Sâm"
1097,100,SN832DAGNVANSAM,lỗi đổi thứ tự các từ,"SN 832, Đặgn Văn Sâm"
1098,100,SN939DANGVANSAM,lỗi sai chính tả,"SN 939, Đặng Văn Sẩm"


In [18]:
#Create a dataframe to contain the calculation data
df_edit=pd.DataFrame(columns=['Street_ID_Add','Street_Name_Add','Street_ID_Train','Street_Name_Train','Edit_distance'])

In [19]:
for add_idx, add_row in e_data_address.iterrows():
    for street_idx, street_row in e_data_training.iterrows():
        # Calculate the edit distance between fb_username and ig_username
        distance = edit_distance(add_row['Street_Name'], street_row['Street_Name'])

        # Create a new DataFrame for the current row
        new_row = pd.DataFrame({
              'Street_ID_Add': [add_row['Street_ID']],
              'Street_Name_Add': [add_row['Street_Name']],
              'Street_Name_Add_Raw': [add_row['Street_Name_Raw']],
              'Street_ID_Train': [street_row['Street_ID']],
              'Street_Name_Train': [street_row['Street_Name']],
              'Street_Name_Train_Raw': [street_row['Street_Name_Raw']],
              'Edit_distance': [distance]
          })

        # Use concat to add the new row to df_edit
        df_edit = pd.concat([df_edit, new_row], ignore_index=True)



In [20]:
df_edit

,Street_ID_Add,Street_Name_Add,Street_ID_Train,Street_Name_Train,Edit_distance,Street_Name_Add_Raw,Street_Name_Train_Raw
0,1,SN19CDANGVANNGU,1,DANGVANNGU,5,"SN 19C, Đặng Văn Ngữ",Đặng Văn Ngữ
1,1,SN19CDANGVANNGU,2,NGUYENVANCU,9,"SN 19C, Đặng Văn Ngữ",Nguyễn Văn Cừ
2,1,SN19CDANGVANNGU,3,TRANHUNGDAO,12,"SN 19C, Đặng Văn Ngữ",Trần Hưng Đạo
3,1,SN19CDANGVANNGU,4,PHAMVANDONG,11,"SN 19C, Đặng Văn Ngữ",Phạm Văn Đồng
4,1,SN19CDANGVANNGU,5,NGUYENTRIPHUONG,14,"SN 19C, Đặng Văn Ngữ",Nguyễn Tri Phương
...,...,...,...,...,...,...,...
109995,100,SN97257ADANGVANSAM,96,PHANDANGLUU,14,"SN 972/57A, Đặng Văn Sâm",Phan Đăng Lưu
109996,100,SN97257ADANGVANSAM,97,HUYNHVANNGHE,15,"SN 972/57A, Đặng Văn Sâm",Huỳnh Văn Nghệ
109997,100,SN97257ADANGVANSAM,98,TRUONGQUOCDUNG,17,"SN 972/57A, Đặng Văn Sâm",Trương Quốc Dung
109998,100,SN97257ADANGVANSAM,99,NGUYENHUUTHO,16,"SN 972/57A, Đặng Văn Sâm",Nguyễn Hữu Thọ


In [21]:
#df_edit.to_csv('edit_distance.csv', encoding='utf-8-sig')

##ACCURACY

In [58]:
# Create DataFrame e_accuracy
e_accuracy = df_edit.copy()
# Create column 'min_edit_distance' (select the smallest edit_distance)
e_accuracy['min_edit_distance'] = 0
for Street_Name_Add in e_accuracy['Street_Name_Add'].unique():
    min_distance = e_accuracy[e_accuracy['Street_Name_Add'] == Street_Name_Add]['Edit_distance'].min()
    e_accuracy.loc[(e_accuracy['Street_Name_Add'] == Street_Name_Add) & (e_accuracy['Edit_distance'] == min_distance), 'min_edit_distance'] = 1
# Mark 1 if 'Street_ID_Add' equals 'edit_distance' and 'min_edit_distance_random' = 1
e_accuracy['addressid_equals_edit_dist'] = 0
e_accuracy.loc[(e_accuracy['Street_ID_Add'] == e_accuracy['Street_ID_Train']) & (e_accuracy['min_edit_distance'] == 1), 'addressid_equals_edit_dist'] = 1

In [60]:
result_edit= e_accuracy['addressid_equals_edit_dist'].sum()/1100*100
print("Accuracy: ",result_edit, " %")

Accuracy:  95.0909090909091  %


Output

In [61]:
a1=e_accuracy[(e_accuracy['min_edit_distance'] == 1)&(e_accuracy['Street_ID_Add']==e_accuracy['Street_ID_Train'])]
a2=e_accuracy[(e_accuracy['min_edit_distance'] == 0)&(e_accuracy['Street_ID_Add']==e_accuracy['Street_ID_Train'])]
a = pd.concat([a1, a2], ignore_index=True)
# sắp xếp theo Street_ID_Add
a = a.sort_values(by=['Street_ID_Add'])
a

,Street_ID_Add,Street_Name_Add,Street_ID_Train,Street_Name_Train,Edit_distance,Street_Name_Add_Raw,Street_Name_Train_Raw,min_edit_distance,addressid_equals_edit_dist
0,1,SN19CDANGVANNGU,1,DANGVANNGU,5,"SN 19C, Đặng Văn Ngữ",Đặng Văn Ngữ,1,1
1,1,SN364DANGAVNNGU,1,DANGVANNGU,7,"SN 364, Đặng ăVn Ngữ",Đặng Văn Ngữ,1,1
2,1,SN41BNAENGVAENNGOO,1,DANGVANNGU,10,"SN 41B, Ñaëng Vaên Ngöõ",Đặng Văn Ngữ,1,1
3,1,SN526CDANGVANNGU,1,DANGVANNGU,6,"SN 526C, Đặng Văn Ngữ",Đặng Văn Ngữ,1,1
4,1,SN601DANGVAN,1,DANGVANNGU,8,"SN 601, Đặng Văn",Đặng Văn Ngữ,1,1
...,...,...,...,...,...,...,...,...,...
1039,100,SN593DANGSAM,100,DANGVANSAM,7,"SN 593, Đặng Sâm",Đặng Văn Sâm,1,1
1038,100,SN342ADNGVANSAM,100,DANGVANSAM,6,"SN 342, ặĐng Văn Sâm",Đặng Văn Sâm,1,1
1037,100,SN26427NHA2TANGDANGVANSAM,100,DANGVANSAM,15,"SN 264/27, Nhà 2 tầng Đặng Văn Sậm",Đặng Văn Sâm,1,1
1035,100,SN21712DANGSAM,100,DANGVANSAM,9,"SN 217/12, Đặng Sẩm",Đặng Văn Sâm,1,1


In [62]:
a.to_csv('edit_distance_output.csv', encoding='utf-8-sig')

#Word embedding

##E5-small-V2 MODEL

Copy data

In [30]:
e5_data_add=data_address.copy()
e5_data_training=data_street_training.copy()

In [31]:
e5_data_add

,Street_ID,Street_Name,Error_Type,Street_Name_Raw
0,1,SN 19C DANG VAN NGU,không lỗi,"SN 19C, Đặng Văn Ngữ"
1,1,SN 364 DANG AVN NGU,lỗi đổi thứ tự các từ,"SN 364, Đặng ăVn Ngữ"
2,1,SN 41B NAENG VAEN NGOO,lỗi font,"SN 41B, Ñaëng Vaên Ngöõ"
3,1,SN 526C DANG VAN NGU,lỗi thiếu từ + đổi thứ tự các từ,"SN 526C, Đặng Văn Ngữ"
4,1,SN 601 DANG VAN,lỗi sai chính tả + thiếu từ,"SN 601, Đặng Văn"
...,...,...,...,...
1095,100,SN 723C NAENG VAEN SAAM,lỗi font,"SN 723C, Ñaëng Vaên Saâm"
1096,100,SN 81057 NHA TRONG DANG VAN SAM,lỗi thêm từ gây nhiễu,"SN 810/57, Nhà trống Đặng Văn Sâm"
1097,100,SN 832 DAGN VAN SAM,lỗi đổi thứ tự các từ,"SN 832, Đặgn Văn Sâm"
1098,100,SN 939 DANG VAN SAM,lỗi sai chính tả,"SN 939, Đặng Văn Sẩm"


In [32]:
e5_data_training

,Street_ID,Street_Name,Street_Name_Raw
0,1,DANG VAN NGU,Đặng Văn Ngữ
1,2,NGUYEN VAN CU,Nguyễn Văn Cừ
2,3,TRAN HUNG DAO,Trần Hưng Đạo
3,4,PHAM VAN DONG,Phạm Văn Đồng
4,5,NGUYEN TRI PHUONG,Nguyễn Tri Phương
...,...,...,...
95,96,PHAN DANG LUU,Phan Đăng Lưu
96,97,HUYNH VAN NGHE,Huỳnh Văn Nghệ
97,98,TRUONG QUOC DUNG,Trương Quốc Dung
98,99,NGUYEN HUU THO,Nguyễn Hữu Thọ


###MODELING

In [33]:
we_e5_model= SentenceTransformer('intfloat/e5-small-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [34]:
e5_add_list=e5_data_add['Street_Name'].tolist()
e5_training_list=e5_data_training['Street_Name'].tolist()
e5_embeddings_add = we_e5_model.encode(e5_add_list, convert_to_tensor=True)
e5_embeddings_training = we_e5_model.encode(e5_training_list, convert_to_tensor=True)
#convert to numpy array for convenient storage
e5_embeddings_add = e5_embeddings_add.detach().numpy()
e5_embeddings_training = e5_embeddings_training.detach().numpy()
#convert the embedding vectors representing the keywords into lists to store into the corresponding "text_vector" columns in the two DataFrames
e5_data_add['text_vector'] = e5_embeddings_add.tolist()
e5_data_training['text_vector'] = e5_embeddings_training.tolist()


In [35]:
e5_data_add

,Street_ID,Street_Name,Error_Type,Street_Name_Raw,text_vector
0,1,SN 19C DANG VAN NGU,không lỗi,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0..."
1,1,SN 364 DANG AVN NGU,lỗi đổi thứ tự các từ,"SN 364, Đặng ăVn Ngữ","[-0.09630431234836578, 0.04084962233901024, 0...."
2,1,SN 41B NAENG VAEN NGOO,lỗi font,"SN 41B, Ñaëng Vaên Ngöõ","[-0.11035669595003128, 0.06765998899936676, 0...."
3,1,SN 526C DANG VAN NGU,lỗi thiếu từ + đổi thứ tự các từ,"SN 526C, Đặng Văn Ngữ","[-0.11222200095653534, 0.05837829038500786, 0...."
4,1,SN 601 DANG VAN,lỗi sai chính tả + thiếu từ,"SN 601, Đặng Văn","[-0.1345066875219345, 0.03675467148423195, 0.0..."
...,...,...,...,...,...
1095,100,SN 723C NAENG VAEN SAAM,lỗi font,"SN 723C, Ñaëng Vaên Saâm","[-0.10876761376857758, 0.04505181312561035, 0...."
1096,100,SN 81057 NHA TRONG DANG VAN SAM,lỗi thêm từ gây nhiễu,"SN 810/57, Nhà trống Đặng Văn Sâm","[-0.10271857678890228, 0.06613828241825104, 0...."
1097,100,SN 832 DAGN VAN SAM,lỗi đổi thứ tự các từ,"SN 832, Đặgn Văn Sâm","[-0.10491720587015152, 0.043730538338422775, 0..."
1098,100,SN 939 DANG VAN SAM,lỗi sai chính tả,"SN 939, Đặng Văn Sẩm","[-0.09822916984558105, 0.030959758907556534, 0..."


In [36]:
e5_data_training

,Street_ID,Street_Name,Street_Name_Raw,text_vector
0,1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02..."
1,2,NGUYEN VAN CU,Nguyễn Văn Cừ,"[-0.08101893961429596, 0.0592430979013443, 0.0..."
2,3,TRAN HUNG DAO,Trần Hưng Đạo,"[-0.07119498401880264, 0.08053550124168396, 0...."
3,4,PHAM VAN DONG,Phạm Văn Đồng,"[-0.10132669657468796, 0.08450060337781906, 0...."
4,5,NGUYEN TRI PHUONG,Nguyễn Tri Phương,"[-0.04958999529480934, 0.08231433480978012, 0...."
...,...,...,...,...
95,96,PHAN DANG LUU,Phan Đăng Lưu,"[-0.09488974511623383, 0.048026930540800095, 0..."
96,97,HUYNH VAN NGHE,Huỳnh Văn Nghệ,"[-0.07475990802049637, 0.0443415492773056, 0.0..."
97,98,TRUONG QUOC DUNG,Trương Quốc Dung,"[-0.06661833822727203, 0.07228459417819977, 0...."
98,99,NGUYEN HUU THO,Nguyễn Hữu Thọ,"[-0.0718488022685051, 0.06744851171970367, 0.0..."


In [37]:
# calculate cosine similarity between 2 data
data_e5_cos = []
for add in range(len(e5_data_add)):
    for training in range(len(e5_data_training)):
            cosine = F.cosine_similarity(torch.tensor(e5_data_add["text_vector"][add]), torch.tensor(e5_data_training["text_vector"][training]), dim=-1).item()
            data_e5_cos.append({
                "Street_ID_Add": e5_data_add["Street_ID"][add],
                "Street_Name_Add": e5_data_add["Street_Name"][add],
                'Street_Name_Add_Raw': e5_data_add["Street_Name_Raw"][add],
                "text_vector_address": e5_data_add["text_vector"][add],
                "Street_ID_Train": e5_data_training["Street_ID"][training],
                "Street_Name_Train": e5_data_training["Street_Name"][training],
                'Street_Name_Train_Raw': e5_data_training["Street_Name_Raw"][training],
                "text_vector_training": e5_data_training["text_vector"][training],
                "cosine": cosine
            })




In [38]:
data_e5_cos=pd.DataFrame(data_e5_cos)
data_e5_cos

,Street_ID_Add,Street_Name_Add,Street_Name_Add_Raw,text_vector_address,Street_ID_Train,Street_Name_Train,Street_Name_Train_Raw,text_vector_training,cosine
0,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02...",0.915395
1,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",2,NGUYEN VAN CU,Nguyễn Văn Cừ,"[-0.08101893961429596, 0.0592430979013443, 0.0...",0.809017
2,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",3,TRAN HUNG DAO,Trần Hưng Đạo,"[-0.07119498401880264, 0.08053550124168396, 0....",0.785746
3,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",4,PHAM VAN DONG,Phạm Văn Đồng,"[-0.10132669657468796, 0.08450060337781906, 0....",0.816895
4,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",5,NGUYEN TRI PHUONG,Nguyễn Tri Phương,"[-0.04958999529480934, 0.08231433480978012, 0....",0.759713
...,...,...,...,...,...,...,...,...,...
109995,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",96,PHAN DANG LUU,Phan Đăng Lưu,"[-0.09488974511623383, 0.048026930540800095, 0...",0.827686
109996,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",97,HUYNH VAN NGHE,Huỳnh Văn Nghệ,"[-0.07475990802049637, 0.0443415492773056, 0.0...",0.762440
109997,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",98,TRUONG QUOC DUNG,Trương Quốc Dung,"[-0.06661833822727203, 0.07228459417819977, 0....",0.792728
109998,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",99,NGUYEN HUU THO,Nguyễn Hữu Thọ,"[-0.0718488022685051, 0.06744851171970367, 0.0...",0.768609


###ACCURACY

In [39]:
e5_accuracy=data_e5_cos.copy()
# Create a 'max_cosine' column (pick out the largest cosine)
e5_accuracy['max_cosine'] = 0
for Street_Name_Add in e5_accuracy['Street_Name_Add'].unique():
    max_cosine = e5_accuracy[e5_accuracy['Street_Name_Add'] == Street_Name_Add]['cosine'].max()
    e5_accuracy.loc[(e5_accuracy['Street_Name_Add'] == Street_Name_Add) & (e5_accuracy['cosine'] == max_cosine), 'max_cosine'] = 1

# Mark 1 if 'add_userid' equals 'training_userid' and 'max_cosine_random' = 1
e5_accuracy['equals_id'] = 0
e5_accuracy.loc[(e5_accuracy['Street_ID_Add'] == e5_accuracy['Street_ID_Train']) & (e5_accuracy['max_cosine'] == 1), 'equals_id'] = 1
e5_accuracy


,Street_ID_Add,Street_Name_Add,Street_Name_Add_Raw,text_vector_address,Street_ID_Train,Street_Name_Train,Street_Name_Train_Raw,text_vector_training,cosine,max_cosine,equals_id
0,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02...",0.915395,1,1
1,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",2,NGUYEN VAN CU,Nguyễn Văn Cừ,"[-0.08101893961429596, 0.0592430979013443, 0.0...",0.809017,0,0
2,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",3,TRAN HUNG DAO,Trần Hưng Đạo,"[-0.07119498401880264, 0.08053550124168396, 0....",0.785746,0,0
3,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",4,PHAM VAN DONG,Phạm Văn Đồng,"[-0.10132669657468796, 0.08450060337781906, 0....",0.816895,0,0
4,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",5,NGUYEN TRI PHUONG,Nguyễn Tri Phương,"[-0.04958999529480934, 0.08231433480978012, 0....",0.759713,0,0
...,...,...,...,...,...,...,...,...,...,...,...
109995,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",96,PHAN DANG LUU,Phan Đăng Lưu,"[-0.09488974511623383, 0.048026930540800095, 0...",0.827686,0,0
109996,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",97,HUYNH VAN NGHE,Huỳnh Văn Nghệ,"[-0.07475990802049637, 0.0443415492773056, 0.0...",0.762440,0,0
109997,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",98,TRUONG QUOC DUNG,Trương Quốc Dung,"[-0.06661833822727203, 0.07228459417819977, 0....",0.792728,0,0
109998,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",99,NGUYEN HUU THO,Nguyễn Hữu Thọ,"[-0.0718488022685051, 0.06744851171970367, 0.0...",0.768609,0,0


In [40]:
# print table where equals_id=1 and max_cosine=1
a=e5_accuracy[(e5_accuracy['max_cosine'] == 1)&(e5_accuracy['equals_id'] == 1)]
a


,Street_ID_Add,Street_Name_Add,Street_Name_Add_Raw,text_vector_address,Street_ID_Train,Street_Name_Train,Street_Name_Train_Raw,text_vector_training,cosine,max_cosine,equals_id
0,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02...",0.915395,1,1
100,1,SN 364 DANG AVN NGU,"SN 364, Đặng ăVn Ngữ","[-0.09630431234836578, 0.04084962233901024, 0....",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02...",0.868838,1,1
300,1,SN 526C DANG VAN NGU,"SN 526C, Đặng Văn Ngữ","[-0.11222200095653534, 0.05837829038500786, 0....",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02...",0.896344,1,1
500,1,SN 68350 DANG VAN NGU,"SN 683/50, Đặng Văn Ngữ","[-0.10004647821187973, 0.02371145226061344, 0....",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02...",0.901588,1,1
600,1,SN 69569 KHONG XAC DINH DANG VAN NGU,"SN 695/69, Không xác định Đặng Văn Ngữ","[-0.10896216332912445, 0.016295472159981728, 0...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02...",0.861359,1,1
...,...,...,...,...,...,...,...,...,...,...,...
109499,100,SN 693 DANG VAN SAM,"SN 693, Đặng Văn Sâm","[-0.10973304510116577, 0.016407260671257973, 0...",100,DANG VAN SAM,Đặng Văn Sâm,"[-0.06774939596652985, 0.05358193442225456, 0....",0.919013,1,1
109699,100,SN 81057 NHA TRONG DANG VAN SAM,"SN 810/57, Nhà trống Đặng Văn Sâm","[-0.10271857678890228, 0.06613828241825104, 0....",100,DANG VAN SAM,Đặng Văn Sâm,"[-0.06774939596652985, 0.05358193442225456, 0....",0.867965,1,1
109799,100,SN 832 DAGN VAN SAM,"SN 832, Đặgn Văn Sâm","[-0.10491720587015152, 0.043730538338422775, 0...",100,DANG VAN SAM,Đặng Văn Sâm,"[-0.06774939596652985, 0.05358193442225456, 0....",0.879223,1,1
109899,100,SN 939 DANG VAN SAM,"SN 939, Đặng Văn Sẩm","[-0.09822916984558105, 0.030959758907556534, 0...",100,DANG VAN SAM,Đặng Văn Sâm,"[-0.06774939596652985, 0.05358193442225456, 0....",0.905510,1,1


In [41]:
result_e5= e5_accuracy['equals_id'].sum()/1100*100
print("Acurracy: ",result_e5, " %")

Acurracy:  86.0909090909091  %


##GTE-LARGE


Copy data

In [42]:
gte_data_add=data_address.copy()
gte_data_training=data_street_training.copy()

###MODELING

In [43]:
gte_model = SentenceTransformer('thenlper/gte-large')

In [44]:
add_gte_list=gte_data_add['Street_Name'].tolist()
training_gte_list=gte_data_training['Street_Name'].tolist()
gte_embeddings_add = gte_model.encode(add_gte_list, convert_to_tensor=True)
gte_embeddings_training = gte_model.encode(training_gte_list, convert_to_tensor=True)
gte_embeddings_add = gte_embeddings_add.detach().numpy()
gte_embeddings_training = gte_embeddings_training.detach().numpy()
gte_data_add['text_vector'] = gte_embeddings_add.tolist()
gte_data_training['text_vector'] = gte_embeddings_training.tolist()


In [45]:
gte_data_add

,Street_ID,Street_Name,Error_Type,Street_Name_Raw,text_vector
0,1,SN 19C DANG VAN NGU,không lỗi,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,..."
1,1,SN 364 DANG AVN NGU,lỗi đổi thứ tự các từ,"SN 364, Đặng ăVn Ngữ","[0.004170349799096584, -0.011334676295518875, ..."
2,1,SN 41B NAENG VAEN NGOO,lỗi font,"SN 41B, Ñaëng Vaên Ngöõ","[-0.002184184966608882, 0.00436954153701663, -..."
3,1,SN 526C DANG VAN NGU,lỗi thiếu từ + đổi thứ tự các từ,"SN 526C, Đặng Văn Ngữ","[-0.006716499570757151, -0.009630143642425537,..."
4,1,SN 601 DANG VAN,lỗi sai chính tả + thiếu từ,"SN 601, Đặng Văn","[-0.002141285687685013, 0.008678363636136055, ..."
...,...,...,...,...,...
1095,100,SN 723C NAENG VAEN SAAM,lỗi font,"SN 723C, Ñaëng Vaên Saâm","[-0.006809805985540152, 0.011011254042387009, ..."
1096,100,SN 81057 NHA TRONG DANG VAN SAM,lỗi thêm từ gây nhiễu,"SN 810/57, Nhà trống Đặng Văn Sâm","[-0.01720387302339077, 0.016107188537716866, -..."
1097,100,SN 832 DAGN VAN SAM,lỗi đổi thứ tự các từ,"SN 832, Đặgn Văn Sâm","[0.0058784205466508865, 0.02194984257221222, 0..."
1098,100,SN 939 DANG VAN SAM,lỗi sai chính tả,"SN 939, Đặng Văn Sẩm","[0.02839365229010582, 0.025042040273547173, -0..."


In [46]:
gte_data_training

,Street_ID,Street_Name,Street_Name_Raw,text_vector
0,1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,..."
1,2,NGUYEN VAN CU,Nguyễn Văn Cừ,"[-0.005895429756492376, 0.005340247415006161, ..."
2,3,TRAN HUNG DAO,Trần Hưng Đạo,"[-0.028865694999694824, -0.038922276347875595,..."
3,4,PHAM VAN DONG,Phạm Văn Đồng,"[0.0032692996319383383, -0.014205255545675755,..."
4,5,NGUYEN TRI PHUONG,Nguyễn Tri Phương,"[-0.03897061198949814, -0.013363678008317947, ..."
...,...,...,...,...
95,96,PHAN DANG LUU,Phan Đăng Lưu,"[-0.024394823238253593, -0.023049095645546913,..."
96,97,HUYNH VAN NGHE,Huỳnh Văn Nghệ,"[-0.012660451233386993, -0.004363514017313719,..."
97,98,TRUONG QUOC DUNG,Trương Quốc Dung,"[-0.016411006450653076, -0.01799963042140007, ..."
98,99,NGUYEN HUU THO,Nguyễn Hữu Thọ,"[-0.004869547672569752, 0.0011099106632173061,..."


In [47]:
# calculate cosine similarity between 2 data
data_gte_cos = []
for add in range(len(gte_data_add)):
    for training in range(len(gte_data_training)):
            cosine = F.cosine_similarity(torch.tensor(gte_data_add["text_vector"][add]), torch.tensor(gte_data_training["text_vector"][training]), dim=-1).item()
            data_gte_cos.append({
                "Street_ID_Add": gte_data_add["Street_ID"][add],
                "Street_Name_Add": gte_data_add["Street_Name"][add],
                "Street_Name_Add_Raw": gte_data_add["Street_Name_Raw"][add],
                "text_vector_address": gte_data_add["text_vector"][add],
                "Street_ID_Train": gte_data_training["Street_ID"][training],
                "Street_Name_Train": gte_data_training["Street_Name"][training],
                "Street_Name_Train_Raw": gte_data_training["Street_Name_Raw"][training],
                "text_vector_training": gte_data_training["text_vector"][training],
                "cosine": cosine
            })



In [48]:
data_gte_cos=pd.DataFrame(data_gte_cos)
data_gte_cos

,Street_ID_Add,Street_Name_Add,Street_Name_Add_Raw,text_vector_address,Street_ID_Train,Street_Name_Train,Street_Name_Train_Raw,text_vector_training,cosine
0,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,...",0.920342
1,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",2,NGUYEN VAN CU,Nguyễn Văn Cừ,"[-0.005895429756492376, 0.005340247415006161, ...",0.851131
2,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",3,TRAN HUNG DAO,Trần Hưng Đạo,"[-0.028865694999694824, -0.038922276347875595,...",0.797677
3,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",4,PHAM VAN DONG,Phạm Văn Đồng,"[0.0032692996319383383, -0.014205255545675755,...",0.815442
4,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",5,NGUYEN TRI PHUONG,Nguyễn Tri Phương,"[-0.03897061198949814, -0.013363678008317947, ...",0.823922
...,...,...,...,...,...,...,...,...,...
109995,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",96,PHAN DANG LUU,Phan Đăng Lưu,"[-0.024394823238253593, -0.023049095645546913,...",0.833848
109996,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",97,HUYNH VAN NGHE,Huỳnh Văn Nghệ,"[-0.012660451233386993, -0.004363514017313719,...",0.790913
109997,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",98,TRUONG QUOC DUNG,Trương Quốc Dung,"[-0.016411006450653076, -0.01799963042140007, ...",0.790205
109998,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",99,NGUYEN HUU THO,Nguyễn Hữu Thọ,"[-0.004869547672569752, 0.0011099106632173061,...",0.799008


###ACCURACY

In [49]:
gte_accuracy=data_gte_cos.copy()
# Create a 'max_cosine' column (pick out the largest cosine)
gte_accuracy['max_cosine'] = 0
for Street_Name_Add in gte_accuracy['Street_Name_Add'].unique():
    max_cosine = gte_accuracy[gte_accuracy['Street_Name_Add'] == Street_Name_Add]['cosine'].max()
    gte_accuracy.loc[(gte_accuracy['Street_Name_Add'] == Street_Name_Add) & (gte_accuracy['cosine'] == max_cosine), 'max_cosine'] = 1

# Mark 1 if 'add_userid' equals 'training_userid' and 'max_cosine_random' = 1
gte_accuracy['equals_id'] = 0
gte_accuracy.loc[(gte_accuracy['Street_ID_Add'] == gte_accuracy['Street_ID_Train']) & (gte_accuracy['max_cosine'] == 1), 'equals_id'] = 1
gte_accuracy


,Street_ID_Add,Street_Name_Add,Street_Name_Add_Raw,text_vector_address,Street_ID_Train,Street_Name_Train,Street_Name_Train_Raw,text_vector_training,cosine,max_cosine,equals_id
0,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,...",0.920342,1,1
1,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",2,NGUYEN VAN CU,Nguyễn Văn Cừ,"[-0.005895429756492376, 0.005340247415006161, ...",0.851131,0,0
2,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",3,TRAN HUNG DAO,Trần Hưng Đạo,"[-0.028865694999694824, -0.038922276347875595,...",0.797677,0,0
3,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",4,PHAM VAN DONG,Phạm Văn Đồng,"[0.0032692996319383383, -0.014205255545675755,...",0.815442,0,0
4,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",5,NGUYEN TRI PHUONG,Nguyễn Tri Phương,"[-0.03897061198949814, -0.013363678008317947, ...",0.823922,0,0
...,...,...,...,...,...,...,...,...,...,...,...
109995,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",96,PHAN DANG LUU,Phan Đăng Lưu,"[-0.024394823238253593, -0.023049095645546913,...",0.833848,0,0
109996,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",97,HUYNH VAN NGHE,Huỳnh Văn Nghệ,"[-0.012660451233386993, -0.004363514017313719,...",0.790913,0,0
109997,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",98,TRUONG QUOC DUNG,Trương Quốc Dung,"[-0.016411006450653076, -0.01799963042140007, ...",0.790205,0,0
109998,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",99,NGUYEN HUU THO,Nguyễn Hữu Thọ,"[-0.004869547672569752, 0.0011099106632173061,...",0.799008,0,0


In [50]:
a=gte_accuracy[(gte_accuracy['max_cosine'] == 1)&(gte_accuracy['equals_id'] == 1)]
a

,Street_ID_Add,Street_Name_Add,Street_Name_Add_Raw,text_vector_address,Street_ID_Train,Street_Name_Train,Street_Name_Train_Raw,text_vector_training,cosine,max_cosine,equals_id
0,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,...",0.920342,1,1
100,1,SN 364 DANG AVN NGU,"SN 364, Đặng ăVn Ngữ","[0.004170349799096584, -0.011334676295518875, ...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,...",0.866095,1,1
300,1,SN 526C DANG VAN NGU,"SN 526C, Đặng Văn Ngữ","[-0.006716499570757151, -0.009630143642425537,...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,...",0.894427,1,1
500,1,SN 68350 DANG VAN NGU,"SN 683/50, Đặng Văn Ngữ","[-0.0010878759203478694, -0.014489724300801754...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,...",0.914009,1,1
600,1,SN 69569 KHONG XAC DINH DANG VAN NGU,"SN 695/69, Không xác định Đặng Văn Ngữ","[-9.865411993814632e-05, 0.009861091151833534,...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,...",0.860435,1,1
...,...,...,...,...,...,...,...,...,...,...,...
109499,100,SN 693 DANG VAN SAM,"SN 693, Đặng Văn Sâm","[0.017218008637428284, 0.015009158290922642, -...",100,DANG VAN SAM,Đặng Văn Sâm,"[0.003528919070959091, -0.008171512745320797, ...",0.909962,1,1
109699,100,SN 81057 NHA TRONG DANG VAN SAM,"SN 810/57, Nhà trống Đặng Văn Sâm","[-0.01720387302339077, 0.016107188537716866, -...",100,DANG VAN SAM,Đặng Văn Sâm,"[0.003528919070959091, -0.008171512745320797, ...",0.903749,1,1
109799,100,SN 832 DAGN VAN SAM,"SN 832, Đặgn Văn Sâm","[0.0058784205466508865, 0.02194984257221222, 0...",100,DANG VAN SAM,Đặng Văn Sâm,"[0.003528919070959091, -0.008171512745320797, ...",0.849970,1,1
109899,100,SN 939 DANG VAN SAM,"SN 939, Đặng Văn Sẩm","[0.02839365229010582, 0.025042040273547173, -0...",100,DANG VAN SAM,Đặng Văn Sâm,"[0.003528919070959091, -0.008171512745320797, ...",0.909785,1,1


In [51]:
result_gte= gte_accuracy['equals_id'].sum()/1100*100
print("Acurracy: ",result_gte, " %")

Acurracy:  88.63636363636364  %


#COMPARE

In [52]:
df_compare = pd.DataFrame([[result_edit], [result_e5], [result_gte]],
                         index=['Edit distance', 'Word embedding with e5 model', 'Word embedding with gte model'],
                         columns=['Accuracy'])
df_compare

,Accuracy
Edit distance,95.090909
Word embedding with e5 model,86.090909
Word embedding with gte model,88.636364


**Thus:**
* Edit Distance is the best method for identifying street names when small character differences are the primary distinction.
* Word embeddings are less effective in this case, likely because they focus more on semantic relationships, which are less relevant for street names.

##nháp


In [ ]:
import random
import pandas as pd
accent_mapping = {
    'á': ['à', 'ả', 'ã', 'ạ'], 'à': ['á', 'ả', 'ã', 'ạ'], 'ả': ['á', 'à', 'ã', 'ạ'], 'ã': ['á', 'à', 'ả', 'ạ'], 'ạ': ['á', 'à', 'ả', 'ã'],
    'é': ['è', 'ẻ', 'ẽ', 'ẹ'], 'è': ['é', 'ẻ', 'ẽ', 'ẹ'], 'ẻ': ['é', 'è', 'ẽ', 'ẹ'], 'ẽ': ['é', 'è', 'ẻ', 'ẹ'], 'ẹ': ['é', 'è', 'ẻ', 'ẽ'],
    'ó': ['ò', 'ỏ', 'õ', 'ọ'], 'ò': ['ó', 'ỏ', 'õ', 'ọ'], 'ỏ': ['ó', 'ò', 'õ', 'ọ'], 'õ': ['ó', 'ò', 'ỏ', 'ọ'], 'ọ': ['ó', 'ò', 'ỏ', 'õ'],
    'ú': ['ù', 'ủ', 'ũ', 'ụ'], 'ù': ['ú', 'ủ', 'ũ', 'ụ'], 'ủ': ['ú', 'ù', 'ũ', 'ụ'], 'ũ': ['ú', 'ù', 'ủ', 'ụ'], 'ụ': ['ú', 'ù', 'ủ', 'ũ'],
    'í': ['ì', 'ỉ', 'ĩ', 'ị'], 'ì': ['í', 'ỉ', 'ĩ', 'ị'], 'ỉ': ['í', 'ì', 'ĩ', 'ị'], 'ĩ': ['í', 'ì', 'ỉ', 'ị'], 'ị': ['í', 'ì', 'ỉ', 'ĩ'],
    'ý': ['ỳ', 'ỷ', 'ỹ', 'ỵ'], 'ỳ': ['ý', 'ỷ', 'ỹ', 'ỵ'], 'ỷ': ['ý', 'ỳ', 'ỹ', 'ỵ'], 'ỹ': ['ý', 'ỳ', 'ỷ', 'ỵ'], 'ỵ': ['ý', 'ỳ', 'ỷ', 'ỹ'],
    'â': ['ấ', 'ầ', 'ẩ', 'ẫ', 'ậ'], 'ấ': ['â', 'ầ', 'ẩ', 'ẫ', 'ậ'], 'ầ': ['â', 'ấ', 'ẩ', 'ẫ', 'ậ'], 'ẩ': ['â', 'ấ', 'ầ', 'ẫ', 'ậ'],
    'ẫ': ['â', 'ấ', 'ầ', 'ẩ', 'ậ'], 'ậ': ['â', 'ấ', 'ầ', 'ẩ', 'ẫ'], 'ê': ['ế', 'ề', 'ể', 'ễ', 'ệ'], 'ế': ['ê', 'ề', 'ể', 'ễ', 'ệ'],
    'ề': ['ê', 'ế', 'ể', 'ễ', 'ệ'], 'ể': ['ê', 'ế', 'ề', 'ễ', 'ệ'], 'ễ': ['ê', 'ế', 'ề', 'ể', 'ệ'], 'ệ': ['ê', 'ế', 'ề', 'ể', 'ễ'],
    'ô': ['ố', 'ồ', 'ổ', 'ỗ', 'ộ'], 'ố': ['ô', 'ồ', 'ổ', 'ỗ', 'ộ'], 'ồ': ['ô', 'ố', 'ổ', 'ỗ', 'ộ'], 'ổ': ['ô', 'ố', 'ồ', 'ỗ', 'ộ'],
    'ỗ': ['ô', 'ố', 'ồ', 'ổ', 'ộ'], 'ộ': ['ô', 'ố', 'ồ', 'ổ', 'ỗ'], 'ơ': ['ớ', 'ờ', 'ở', 'ỡ', 'ợ'], 'ớ': ['ơ', 'ờ', 'ở', 'ỡ', 'ợ'],
    'ờ': ['ơ', 'ớ', 'ở', 'ỡ', 'ợ'], 'ở': ['ơ', 'ớ', 'ờ', 'ỡ', 'ợ'], 'ỡ': ['ơ', 'ớ', 'ờ', 'ở', 'ợ'], 'ợ': ['ơ', 'ớ', 'ờ', 'ở', 'ỡ'],
    'ư': ['ứ', 'ừ', 'ử', 'ữ', 'ự'], 'ứ': ['ư', 'ừ', 'ử', 'ữ', 'ự'], 'ừ': ['ư', 'ứ', 'ử', 'ữ', 'ự'], 'ử': ['ư', 'ứ', 'ừ', 'ữ', 'ự'],
    'ữ': ['ư', 'ứ', 'ừ', 'ử', 'ự'], 'ự': ['ư', 'ứ', 'ừ', 'ử', 'ữ']
}

# Function to introduce spelling errors
def introduce_spelling_errors(street_name):
    modified_name = list(street_name)
    for i, char in enumerate(modified_name):
        if char in accent_mapping:
            modified_name[i] = random.choice(accent_mapping[char])
    return ''.join(modified_name)
# Adjust the function to create a variety of character-level errors for "lỗi đổi thứ tự các từ"
def introduce_error_v2(street_name, error_type):
    noise_words = ["Ngõ", "Hẻm", "Phố", "Ấp", "Phường", "Không biết", "Không xác định", "Nhà trống", "Nhà 2 tầng", "CSH"]
    if error_type == "lỗi font":
        return street_name.replace("a", "â").replace("o", "ô")  # Example for font error
    elif error_type == "lỗi sai chính tả":
       return introduce_spelling_errors(street_name)
    elif error_type == "lỗi đổi thứ tự các từ":
        # Introduce character-level errors as described
        words = street_name.split()
        random_word_index = random.randint(0, len(words) - 1)
        word = list(words[random_word_index])
        if len(word) > 1:
            # Swap two adjacent characters randomly within the selected word
            i = random.randint(0, len(word) - 2)
            word[i], word[i + 1] = word[i + 1], word[i]
            words[random_word_index] = "".join(word)
        return " ".join(words)
    elif error_type == "lỗi thêm từ gây nhiễu":
        return random.choice(noise_words) + " " + street_name
    elif error_type == "lỗi sai chính tả + thêm từ gây nhiễu":
        return random.choice(noise_words) + " " + introduce_spelling_errors(street_name)
    elif error_type == "lỗi thiếu từ":
        words = street_name.split()
        # Remove a word randomly
        if len(words) > 1:
            words.pop(random.randint(0, len(words)-1))
        return ' '.join(words)
    elif error_type == "lỗi sai chính tả + thiếu từ":
        street_name=introduce_spelling_errors(street_name)
        words = street_name.split()
        # Remove a word randomly
        if len(words) > 1:
            words.pop(random.randint(0, len(words)-1))
        return ' '.join(words)
    elif error_type == "lỗi sai chính tả + đổi thứ tự các từ":
        # Introduce character-level errors as described
        introduce_spelling_errors(street_name)
        words = street_name.split()
        random_word_index = random.randint(0, len(words) - 1)
        word = list(words[random_word_index])
        if len(word) > 1:
            # Swap two adjacent characters randomly within the selected word
            i = random.randint(0, len(word) - 2)
            word[i], word[i + 1] = word[i + 1], word[i]
            words[random_word_index] = "".join(word)
        return " ".join(words)


    return street_name

# Regenerate the dataframe with the updated error type logic
data_v2 = []
street_data = [
    (1, 'Đặng Văn Ngữ'),
(2, 'Nguyễn Văn Cừ'),
(3, 'Trần Hưng Đạo'),
(4, 'Phạm Văn Đồng'),
(5, 'Nguyễn Tri Phương'),
(6, 'Lê Văn Sỹ'),
(7, 'Phan Đình Phùng'),
(8, 'Hoàng Văn Thụ'),
(9, 'Lý Thường Kiệt'),
(10, 'Bà Huyện Thanh Quan'),
(11, 'Tôn Đức Thắng'),
(12, 'Nguyễn Văn Trỗi'),
(13, 'Nguyễn Thị Minh Khai'),
(14, 'Võ Văn Tần'),
(15, 'Lê Thánh Tôn'),
(16, 'Đinh Tiên Hoàng'),
(17, 'Nguyễn Hữu Cảnh'),
(18, 'Phan Chu Trinh'),
(19, 'Lý Tự Trọng'),
(20, 'Huỳnh Văn Bánh'),
(21, 'Võ Thị Sáu'),
(22, 'Nguyễn Khắc Viện'),
(23, 'Cách Mạng Tháng Tám'),
(24, 'Đào Duy Anh'),
(25, 'Lý Chính Thắng'),
(26, 'Trần Quang Khải'),
(27, 'Hai Bà Trưng'),
(28, 'Đặng Thái Thân'),
(29, 'Nguyễn Văn Quá'),
(30, 'Nguyễn Chí Thanh'),
(31, 'Trần Khánh Dư'),
(32, 'Hồ Tùng Mậu'),
(33, 'Nguyễn Cư Trinh'),
(34, 'Phan Kế Bính'),
(35, 'Ngô Gia Tự'),
(36, 'Phan Xích Long'),
(37, 'Mai Thị Lựu'),
(38, 'Phan Huy Ích'),
(39, 'Vũ Tông Phan'),
(40, 'Nguyễn Đình Chiểu'),
(41, 'Nguyễn An Ninh'),
(42, 'Nguyễn Hữu Thọ'),
(43, 'Nguyễn Văn Linh'),
(44, 'Lê Đại Hành'),
(45, 'Lê Văn Lương'),
(46, 'Nguyễn Quang Bích'),
(47, 'Hoàng Quốc Việt'),
(48, 'Trần Cao Vân'),
(49, 'Trần Quốc Toản'),
(50, 'Phạm Ngọc Thạch'),
(51, 'Trương Công Định'),
(52, 'Nguyễn Hữu Thọ'),
(53, 'Trần Xuân Soạn'),
(54, 'Nguyễn Thượng Hiền'),
(55, 'Phan Văn Trị'),
(56, 'Nguyễn Văn Thoại'),
(57, 'Nguyễn Kim Hồng'),
(58, 'Lê Trọng Tấn'),
(59, 'Huỳnh Tấn Phát'),
(60, 'Ngô Tất Tố'),
(61, 'Nguyễn Văn Công'),
(62, 'Đinh Bộ Lĩnh'),
(63, 'Phan Thanh Giản'),
(64, 'Huỳnh Tịnh Của'),
(65, 'Nguyễn Thị Định'),
(66, 'Võ Văn Kiệt'),
(67, 'Trần Văn Đang'),
(68, 'Nguyễn Văn Luông'),
(69, 'Bùi Thị Xuân'),
(70, 'Lê Hồng Phong'),
(71, 'Phạm Văn Hai'),
(72, 'Đỗ Xuân Hợp'),
(73, 'Nguyễn Văn Tạo'),
(74, 'Phan Tôn Đức'),
(75, 'Nguyễn Bỉnh Khiêm'),
(76, 'Nguyễn Văn Tứ'),
(77, 'Lê Thanh Nghị'),
(78, 'Phạm Văn Chiêu'),
(79, 'Huỳnh Văn Lũy'),
(80, 'Trần Văn Giàu'),
(81, 'Đinh Công Tráng'),
(82, 'Nguyễn Văn Mại'),
(83, 'Nguyễn Thiện Thuật'),
(84, 'Nguyễn Văn Lộc'),
(85, 'Trần Văn Kỷ'),
(86, 'Võ Thị Sáu'),
(87, 'Nguyễn Khuyến'),
(88, 'Nguyễn Văn Hưởng'),
(89, 'Nguyễn Văn Đậu'),
(90, 'Trương Định'),
(91, 'Lê Văn Tám'),
(92, 'Trần Quốc Hoàn'),
(93, 'Nguyễn Trọng Tuyển'),
(94, 'Đặng Thái Mai'),
(95, 'Nguyễn Phúc Nguyên'),
(96, 'Phan Đăng Lưu'),
(97, 'Huỳnh Văn Nghệ'),
(98, 'Trương Quốc Dung'),
(99, 'Nguyễn Hữu Thọ'),
(100, 'Đặng Văn Sâm')
]
# Define the function to generate random house numbers
def random_house_number():
    """Generates a random house number.
    You can customize the range and format as needed.
    """
    return random.randint(1, 1000)
for street_id, street_name in street_data:
    # Generate two correct entries for each street
    for _ in range(2):
        data_v2.append({
            "street_id": street_id,
            "street_name": street_name,
            "số nhà": random_house_number(),
            "loại lỗi": "không lỗi"
        })
    # Generate five error entries for each street with different types of errors
    error_types = ["lỗi font", "lỗi sai chính tả", "lỗi đổi thứ tự các từ", "lỗi thêm từ gây nhiễu", "lỗi sai chính tả + thêm từ gây nhiễu","lỗi thiếu từ","lỗi sai chính tả + thiếu từ","lỗi sai chính tả + đổi thứ tự các từ","lỗi thiếu từ + đổi thứ tự các từ"]
    for error_type in error_types:
        data_v2.append({
            "street_id": street_id,
            "street_name": introduce_error_v2(street_name, error_type),
            "số nhà": random_house_number(),
            "loại lỗi": error_type
        })

# Convert to DataFrame

df_v2 = pd.DataFrame(data_v2)

# Export to CSV
df_v2.to_csv('dia_chinh_v3.csv', index=False, encoding='utf-8-sig')





In [ ]:
import random

# Tạo danh sách đường và chung cư từ dữ liệu ảnh
ten_duong = [
    "Đỗ Xuân Hợp", "Đình Phong Phú", "Tăng Nhơn Phú", "Tăng Nhơn Phú B",
    "Xa Lộ Hà Nội", "Quốc Lộ 52", "Diệp Minh Tuyền", "Út Trà Ôn",
    "Liên Phường", "Trịnh Công Sơn", "Nguyễn Đình Thi", "Phạm Trọng Cầu",
    "Huy Cận", "Thanh Nga", "Xuân Quỳnh", "Đinh Phong Phú", "Phạm Trọng Cầu",
    "Trần Thị Điệu", "Nguyễn Đình Thi", "Phạm Trọng Cầu"
]

ten_chung_cu = [
    "Chung cư EHome Đông Sài GonF1", "Chung cư Phước Long B", "Chung cư 9 View",
    "Chung cư EHome 2", "Chung cư Gia Hòa", "Chung cư Flora Fuji",
    "Chung cư Phố Đông", "Chung cư 9 View Apartment", "Chung cư EHome 2",
    "Chung cư Phước Long B", "Chung cư Giai Hòa", "Chung cư Flora Fuji",
    "Chung cư 9 View", "Chung cư Phố Đông", "Chung cư EHome", "Chung cư Flora Fuji",
    "Chung cư Gia Hòa", "Chung cư EHome", "Chung cư Phố Đông", "Chung cư Phước Long B"
]

# Hàm tạo lỗi sai chính tả
def generate_typo(text):
    chars = list(text)
    random_idx = random.randint(0, len(chars) - 2)
    chars[random_idx], chars[random_idx + 1] = chars[random_idx + 1], chars[random_idx]
    return ''.join(chars)

# Hàm giả lập lỗi font bằng cách thay đổi một số ký tự
def generate_font_error(text):
    error_chars = {'A': '4', 'O': '0', 'E': '3', 'I': '1', 'S': '5', 'D': 'Ð', 'C': 'Ç'}
    return ''.join(error_chars.get(c, c) for c in text)

# Hàm đảo thứ tự từ
def shuffle_words(text):
    words = text.split()
    random.shuffle(words)
    return ' '.join(words)

# Tạo dữ liệu cho từng loại lỗi và dữ liệu bình thường
data = []

# 40 hàng bình thường
for i in range(40):
    sn = random.randint(1, 999)
    duong = random.choice(ten_duong)
    chung_cu = random.choice(ten_chung_cu)
    data.append({"Số nhà": sn, "Tên đường": duong, "Tên chung cư": chung_cu, "Loại lỗi": "Bình thường"})

# 20 hàng sai chính tả
for i in range(20):
    sn = random.randint(1, 999)
    duong = generate_typo(random.choice(ten_duong))
    chung_cu = generate_typo(random.choice(ten_chung_cu))
    data.append({"Số nhà": sn, "Tên đường": duong, "Tên chung cư": chung_cu, "Loại lỗi": "Sai chính tả"})

# 20 hàng lỗi font
for i in range(20):
    sn = random.randint(1, 999)
    duong = generate_font_error(random.choice(ten_duong))
    chung_cu = generate_font_error(random.choice(ten_chung_cu))
    data.append({"Số nhà": sn, "Tên đường": duong, "Tên chung cư": chung_cu, "Loại lỗi": "Lỗi font"})

# 20 hàng đảo thứ tự từ
for i in range(20):
    sn = random.randint(1, 999)
    duong = shuffle_words(random.choice(ten_duong))
    chung_cu = shuffle_words(random.choice(ten_chung_cu))
    data.append({"Số nhà": sn, "Tên đường": duong, "Tên chung cư": chung_cu, "Loại lỗi": "Đảo thứ tự từ"})

data


[{'Số nhà': 860,
  'Tên đường': 'Thanh Nga',
  'Tên chung cư': 'Chung cư EHome',
  'Loại lỗi': 'Bình thường'},
 {'Số nhà': 234,
  'Tên đường': 'Diệp Minh Tuyền',
  'Tên chung cư': 'Chung cư Flora Fuji',
  'Loại lỗi': 'Bình thường'},
 {'Số nhà': 655,
  'Tên đường': 'Huy Cận',
  'Tên chung cư': 'Chung cư Flora Fuji',
  'Loại lỗi': 'Bình thường'},
 {'Số nhà': 876,
  'Tên đường': 'Phạm Trọng Cầu',
  'Tên chung cư': 'Chung cư Giai Hòa',
  'Loại lỗi': 'Bình thường'},
 {'Số nhà': 190,
  'Tên đường': 'Tăng Nhơn Phú B',
  'Tên chung cư': 'Chung cư Flora Fuji',
  'Loại lỗi': 'Bình thường'},
 {'Số nhà': 550,
  'Tên đường': 'Trần Thị Điệu',
  'Tên chung cư': 'Chung cư Gia Hòa',
  'Loại lỗi': 'Bình thường'},
 {'Số nhà': 621,
  'Tên đường': 'Diệp Minh Tuyền',
  'Tên chung cư': 'Chung cư Phước Long B',
  'Loại lỗi': 'Bình thường'},
 {'Số nhà': 762,
  'Tên đường': 'Nguyễn Đình Thi',
  'Tên chung cư': 'Chung cư Flora Fuji',
  'Loại lỗi': 'Bình thường'},
 {'Số nhà': 821,
  'Tên đường': 'Nguyễn Đình Thi'

In [ ]:
import random

# Tạo danh sách đường và chung cư từ dữ liệu ảnh
ten_duong = pd.read_csv('/content/STREET_ID.csv')
ten_duong = [row[1] for row in ten_duong.values.tolist()]
    # Tiếp tục bổ sung danh sách đường

ten_chung_cu = pd.read_csv('/content/APARTMENT_ID.csv')
ten_chung_cu = [row[1] for row in ten_chung_cu.values.tolist()]
    # Tiếp tục bổ sung danh sách chung cư

# Hàm tạo lỗi sai chính tả
def generate_typo(text):
    chars = list(text)
    random_idx = random.randint(0, len(chars) - 2)
    chars[random_idx], chars[random_idx + 1] = chars[random_idx + 1], chars[random_idx]
    return ''.join(chars)

# Hàm giả lập lỗi font bằng cách thay đổi một số ký tự
def generate_font_error(text):
    error_chars = {'A': '4', 'O': '0', 'E': '3', 'I': '1', 'S': '5', 'D': 'Ð', 'C': 'Ç'}
    return ''.join(error_chars.get(c, c) for c in text)

# Hàm đảo thứ tự từ
def shuffle_words(text):
    words = text.split()
    random.shuffle(words)
    return ' '.join(words)

# Tạo dữ liệu cho từng loại lỗi và dữ liệu bình thường
data = []

# 40 hàng bình thường
for i in range(40):
    sn = random.randint(1, 999)
    #lấy duong_id theo df ten_duong
    duong = random.choice(ten_duong)
    duong_id = ten_duong.index(duong) + 1
    chung_cu = random.choice(ten_chung_cu)
    chung_cu_id = ten_chung_cu.index(chung_cu) + 1
    data.append({"Số nhà": sn, "Tên đường": duong, "duong_id":duong_id ,"Tên chung cư": chung_cu, 'chung_cu_id':chung_cu_id,"Loại lỗi": "Bình thường"})

# 20 hàng sai chính tả
for i in range(20):
    sn = random.randint(1, 999)
    duong = generate_typo(random.choice(ten_duong))
    #lấy duong_id theo df ten_duong mà trước khi lấy random

    duong_id = ten_duong.index(duong) + 1
    chung_cu = generate_typo(random.choice(ten_chung_cu))
    chung_cu_id = ten_chung_cu.index(chung_cu) + 1
    data.append({"Số nhà": sn, "Tên đường": duong, "duong_id":duong_id ,"Tên chung cư": chung_cu, 'chung_cu_id':chung_cu_id, "Loại lỗi": "Sai chính tả"})
    data.app

# 20 hàng lỗi font
for i in range(20):
    sn = random.randint(1, 999)
    duong = generate_font_error(random.choice(ten_duong))
    duong_id = ten_duong.index(duong) + 1
    chung_cu = generate_font_error(random.choice(ten_chung_cu))
    chung_cu_id = ten_chung_cu.index(chung_cu) + 1
    data.append({"Số nhà": sn, "Tên đường": duong, "duong_id":duong_id ,"Tên chung cư": chung_cu, 'chung_cu_id':chung_cu_id, "Loại lỗi": "Lỗi font"})

# 20 hàng đảo thứ tự từ
for i in range(20):
    sn = random.randint(1, 999)
    duong = shuffle_words(random.choice(ten_duong))
    duong_id = ten_duong.index(duong) + 1
    chung_cu = shuffle_words(random.choice(ten_chung_cu))
    chung_cu_id = ten_chung_cu.index(chung_cu) + 1
    data.append({"Số nhà": sn, "Tên đường": duong, "duong_id":duong_id ,"Tên chung cư": chung_cu, 'chung_cu_id':chung_cu_id, "Loại lỗi": "Đảo thứ tự từ"})

data=pd.DataFrame(data)
#lưu csv tiếng việt
data.to_csv('output.csv', index=False, encoding='utf-8-sig')



FileNotFoundError: [Errno 2] No such file or directory: '/content/STREET_ID.csv'

In [ ]:
import csv
import random

# Dữ liệu mô phỏng (chỉ một phần danh sách để minh họa, vui lòng bổ sung đầy đủ cho tập của bạn)
streets =pd.read_csv('/content/STREET_ID.csv')
streets = streets.values.tolist()
    # Tiếp tục bổ sung danh sách đường


apartments = pd.read_csv('/content/APARTMENT_ID.csv')
apartments = apartments.values.tolist()
    # Tiếp tục bổ sung danh sách chung cư

# Danh sách dữ liệu kết quả
data = []

# Hàm tạo lỗi
def create_typo(name):
    if len(name) < 2:
        return name
    i = random.randint(0, len(name) - 2)
    return name[:i] + name[i+1] + name[i] + name[i+2:]

def create_font_error(name):
    replacements = {'Á': 'Á', 'Ô': 'Ô', 'Ê': 'Ê', 'Ã': 'Ã'}
    return ''.join(replacements.get(char, char) for char in name)

def create_order_error(name):
    parts = name.split()
    random.shuffle(parts)
    return ' '.join(parts)

# Tạo hàng dữ liệu với lỗi ngẫu nhiên
def generate_entry(street_tuple, apartment_tuple=None, error_type=None):
    street_id, street_name = street_tuple
    apartment_id, apartment_name = apartment_tuple if apartment_tuple else (None, None)

    if error_type == "typo":
        street_name = create_typo(street_name)
        if apartment_name:
            apartment_name = create_typo(apartment_name)
    elif error_type == "font":
        street_name = create_font_error(street_name)
        if apartment_name:
            apartment_name = create_font_error(apartment_name)
    elif error_type == "order":
        street_name = create_order_error(street_name)
        if apartment_name:
            apartment_name = create_order_error(apartment_name)

    house_number = random.randint(1, 500)

    return [
        street_id,
        apartment_id,
        street_name,
        apartment_name,
        house_number,
        error_type if street_name else None,
        error_type if apartment_name else None
    ]

# Tạo các mục dữ liệu với loại lỗi ngẫu nhiên
error_types = ["typo", "font", "order", None]
random.shuffle(streets)
random.shuffle(apartments)

for i in range(20):  #ấn định số lần để dữ liệu có 5 lỗi mỗi loại cho apartment
    error_type = random.choice(error_types[:-1])
    data.append(generate_entry(random.choice(streets), random.choice(apartments), error_type=error_type))

for i in range(40):  # dữ liệu có 15 lỗi mỗi loại cho streets
    error_type = random.choice(error_types[:-1])
    data.append(generate_entry(random.choice(streets), error_type=error_type))

for i in range(50):  # Hàng không lỗi (ngẫu nhiên có hoặc không có apartment)
    data.append(generate_entry(random.choice(streets), random.choice(apartments)))

for i in range(50):  # Hàng không lỗi (chỉ streets)
    data.append(generate_entry(random.choice(streets)))

# Xuất ra tệp CSV với mã hóa UTF-8
header = ["Street_ID", "Apartment_ID", "Street_Name", "Apartment_Name", "House_Number", "Error_Type_Street", "Error_Type_Apartment"]
with open("output.csv", "w", encoding='utf-8-sig', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(header)
    writer.writerows(data)

print("CSV file generated with mixed errors in UTF-8 encoding.")


In [ ]:
import pandas as pd
import random

# Dữ liệu từ bảng đã cung cấp
data = {
    'STREET_ID': list(range(1,107)),
    'STREET_NAME': ['BÁC ÁI', 'BÌNH CHIỂU', 'BÌNH PHÚ', 'BÌNH QUỚI 2', 'BÙI HỮU NGHĨA', 'CẶP ĐƯỜNG SẮT',
                    'CÂY KEO', 'CHÂN LÝ', 'CHU MẠNH TRINH', 'CHÙA BỬU PHƯỚC', 'CHƯƠNG DƯƠNG', 'CÔNG LÝ',
                    'D1', 'D2', 'D3', 'D4', 'D5', 'DÂN CHỦ', 'ĐẶNG THỊ RÀNH', 'ĐẶNG VĂN BI', 'ĐẤT ĐỎ',
                    'ĐINH THỊ THI', 'ĐINH TIÊN HOÀNG', 'DÒ DƯA', 'ĐOÀN CÔNG HỚN', 'ĐOÀN KẾT', 'DỌC SUỐI NHUM',
                    'ĐỒNG TIẾN', 'DƯƠNG VĂN CAM', 'ĐƯỜNG VÀO TRUNG TÂM DẠY NGHỀ', 'EINSTEIN', 'GÒ DƯA',
                    'HÁN PHÚC', 'HÀN THUYÊN', 'HIỆP BÌNH', 'HỒ VĂN TƯ', 'HÒA BÌNH', 'HOÀNG DIỆU 2', 'HỒNG ĐỨC',
                    'HỮU NGHỊ', 'KHA VẠN CÂN', 'KHA VẠN HẠNH', 'KHỔNG TỬ', 'LAM SƠN', 'LÊ HỒNG PHONG',
                    'LÊ THỊ HOA', 'LÊ VĂN CHÍ', 'LÊ VĂN NINH', 'LÊ VĂN TÁCH', 'LIÊN PHƯỜNG', 'LINH ĐÔNG',
                    'LINH TRUNG', 'LÝ TẾ XUYÊN', 'MINH ĐỨC', 'NGÔ CHÍ', 'NGÔ CHÍ CÔNG', 'NGÔ CHÍ QUỐC',
                    'NGỌC VÂN', 'NGUYỄN BÁ LUẬT', 'NGUYỄN CHÍ QUỐC', 'NGUYỄN PHÚC TÀI', 'NGUYỄN THỊ NHUNG',
                    'NGUYỄN VĂN BÁ', 'NGUYỄN VĂN LỊCH', 'PHẠM VĂN ĐỒNG', 'PHAN HUY CHÚ', 'PHÚ CHÂU',
                    'PHƯỜNG TRƯỜNG THỌ', 'QUỐC LỘ', 'QUỐC LỘ 1', 'QUỐC LỘ 13', 'QUỐC LỘ 139', 'QUỐC LỘ 1A',
                    'QUỐC LỘ 1K', 'RẠCH LÙNG', 'SÔNG ĐÀ', 'SONG HÀNH', 'SUỐI NHUM', 'SỸ LÂM', 'TAM BINH',
                    'TAM BÌNH', 'TAM CHÂU', 'TAM HÀ', 'TAM HẢI', 'TAM PHÚ', 'TÂM TÂM XÃ', 'THỊ DUNG',
                    'THỐNG NHẤT', 'TỈNH LỘ', 'TỈNH LỘ 43', 'TÔ NGỌC VÂN', 'TÔ VĨNH DIỆN', 'TRẦN VĂN NỮA',
                    'TRƯỜNG SƠN', 'TRƯỜNG TRE', 'TRƯƠNG VĂN NGƯ', 'Ụ GHE', 'VĂN NAM', 'VĂN THẠNH',
                    'VĂN THỌ', 'VIỆT THẮNG', 'VÕ HOÀNG NGÂN', 'VÕ TRỨ', 'VÕ VĂN NGÂN', 'XA LỘ HÀ NỘI',
                    'XA LỘ TRƯỜNG SƠN']
}

# Chuyển đổi thành DataFrame
df = pd.DataFrame(data)

# Hàm tạo lỗi chính tả (đảo vị trí 2 ký tự)
def create_typo(text):
    if len(text) > 1:
        idx = random.randint(0, len(text) - 2)
        return text[:idx] + text[idx + 1] + text[idx] + text[idx + 2:]
    return text

# Hàm tạo lỗi font chữ (thay thế một số ký tự bằng ký tự lạ)
def create_font_error(text):
    replacements = {'a': 'á', 'o': 'ô', 'e': 'ê', 'u': 'ư', 'i': 'ì'}
    return ''.join([replacements.get(c, c) for c in text])

# Hàm đảo thứ tự từ
def reverse_words(text):
    words = text.split()
    random.shuffle(words)
    return ' '.join(words)

# Tạo số nhà ngẫu nhiên
def generate_house_number():
    return random.randint(1, 999)

# Tạo bảng với các loại lỗi và hàng bình thường
records = []
error_types = ['typo', 'font', 'reverse', 'normal']

for i in range(424):

    error_type = random.choices(error_types, weights=[106, 106, 106, 106])[0]

    street_row = df.sample(1).iloc[0]

    street_name = street_row['STREET_NAME']

    if error_type == 'typo':
        street_name = create_typo(street_name)
    elif error_type == 'font':
        street_name = create_font_error(street_name)
    elif error_type == 'reverse':
        street_name = reverse_words(street_name)

    house_number = generate_house_number()

    records.append({
        'Street_ID': street_row['STREET_ID'],

        'Street_Name': street_name,

        'House_Number': house_number,
        'Error_Type_Street': error_type if error_type != 'normal' else None,

    })

# Chuyển thành DataFrame
final_df = pd.DataFrame(records)

# Xuất ra file CSV
final_df.to_csv('address_data.csv', index=False, encoding='utf-8-sig')

In [ ]:
import random

# Hàm tạo số nhà ngẫu nhiên theo chuẩn Việt Nam
def generate_house_number():
    # Các mẫu số nhà phổ biến
    patterns = [
        lambda: str(random.randint(1, 999)),                    # Số tự nhiên đơn giản
        lambda: f"{random.randint(1, 999)}{random.choice(['A', 'B', 'C'])}",  # Số với chữ cái đằng sau
        lambda: f"{random.randint(1, 999)}/{random.randint(1, 99)}",          # Số với dấu gạch chéo
        lambda: f"{random.randint(1, 999)}/{random.randint(1, 99)}{random.choice(['A', 'B', 'C'])}"  # Số với gạch chéo và chữ cái
    ]
    # Chọn một mẫu ngẫu nhiên
    return random.choice(patterns)()

# Tạo ra 500 số nhà ngẫu nhiên
house_numbers = [generate_house_number() for _ in range(1100)]

# In danh sách các số nhà
for house_number in house_numbers:
  #in và lưu vào danh sách
    print("'{0}',".format(house_number))


'683/50',
'19C',
'41B',
'809A',
'364',
'695/69',
'943/79',
'904B',
'601',
'78',
'526C',
'252B',
'243B',
'724A',
'700/62',
'198B',
'533/3C',
'492C',
'377C',
'458/60B',
'69/19A',
'732/23',
'132/77',
'975/98B',
'451B',
'880/36C',
'272B',
'582C',
'952/18',
'828/86',
'12',
'683',
'711',
'751/18',
'663/40',
'971/23',
'992/3',
'112',
'724B',
'814',
'507C',
'35/6B',
'277/7',
'222/8A',
'57/72A',
'518C',
'381',
'661',
'649/86',
'46C',
'519',
'16A',
'113/14B',
'238/99B',
'394/77A',
'320/55B',
'721',
'286',
'729C',
'389/62',
'626/52B',
'550/52',
'948A',
'819/1B',
'82/32',
'804/82B',
'798/40C',
'501/74C',
'466/86',
'929/81B',
'413B',
'783B',
'216/24',
'111',
'445/56',
'487/94',
'925B',
'823',
'199A',
'646/18C',
'788',
'206',
'590/30',
'38C',
'950',
'749/1',
'616A',
'327',
'598/66A',
'463/63',
'214/59A',
'827/54',
'36C',
'545A',
'753/79',
'405/53',
'782/14B',
'558/11C',
'486',
'658/71',
'251/27',
'645/9C',
'641B',
'765/75',
'422/97A',
'728',
'787',
'866',
'684',
'233/85B',
'154C',
'109/9',
'234/51C'

In [ ]:
li=['683/50',
'19C',
'41B',
'809A',
'364',
'695/69',
'943/79',
'904B',
'601',
'78',
'526C',
'252B',
'243B',
'724A',
'700/62',
'198B',
'533/3C',
'492C',
'377C',
'458/60B',
'69/19A',
'732/23',
'132/77',
'975/98B',
'451B',
'880/36C',
'272B',
'582C',
'952/18',
'828/86',
'12',
'683',
'711',
'751/18',
'663/40',
'971/23',
'992/3',
'112',
'724B',
'814',
'507C',
'35/6B',
'277/7',
'222/8A',
'57/72A',
'518C',
'381',
'661',
'649/86',
'46C',
'519',
'16A',
'113/14B',
'238/99B',
'394/77A',
'320/55B',
'721',
'286',
'729C',
'389/62',
'626/52B',
'550/52',
'948A',
'819/1B',
'82/32',
'804/82B',
'798/40C',
'501/74C',
'466/86',
'929/81B',
'413B',
'783B',
'216/24',
'111',
'445/56',
'487/94',
'925B',
'823',
'199A',
'646/18C',
'788',
'206',
'590/30',
'38C',
'950',
'749/1',
'616A',
'327',
'598/66A',
'463/63',
'214/59A',
'827/54',
'36C',
'545A',
'753/79',
'405/53',
'782/14B',
'558/11C',
'486',
'658/71',
'251/27',
'645/9C',
'641B',
'765/75',
'422/97A',
'728',
'787',
'866',
'684',
'233/85B',
'154C',
'109/9',
'234/51C',
'974/6',
'112',
'910',
'31',
'948/22',
'718/74A',
'770',
'318B',
'743',
'303/20',
'972/61',
'226/65',
'490B',
'75/30',
'967/90',
'358/71',
'819/98',
'877/19C',
'849/60',
'126/96',
'590',
'412A',
'171/9',
'627/83C',
'354/63',
'240/39A',
'725/86C',
'147',
'585',
'748/3',
'360B',
'554/5C',
'723',
'37',
'530',
'113',
'391',
'777/63',
'504',
'40/31',
'192/86A',
'272/57A',
'404B',
'454',
'380C',
'877/51',
'583/44C',
'48',
'796/91A',
'859/81',
'961/91B',
'938/13B',
'765C',
'19/82',
'71/35C',
'162/79A',
'865/68C',
'448/97B',
'331/36A',
'5/49',
'413/63',
'703/9C',
'53/2',
'625/76',
'328/31',
'429/23',
'957C',
'419/24',
'61',
'221/36',
'133',
'120C',
'992A',
'173',
'119/48C',
'910/86',
'701B',
'342A',
'272',
'766C',
'441/54A',
'857',
'30/69A',
'572/7',
'892',
'658',
'486',
'416/32C',
'100/62',
'763',
'443/41',
'295B',
'842A',
'720',
'717A',
'50',
'884/67',
'8/7',
'168/33B',
'906/67A',
'371B',
'323/45',
'172',
'458A',
'55/20',
'367/36C',
'751/83B',
'291/34B',
'233/73',
'559/90A',
'874/88',
'111C',
'81',
'841/29',
'405/91A',
'980B',
'451/62',
'407A',
'845',
'918/45B',
'811/95A',
'714/21',
'964/7',
'697C',
'748',
'987/41',
'955/97',
'851',
'824',
'994C',
'628A',
'847/85C',
'457/19',
'644/40',
'722/46',
'433C',
'430/29B',
'525C',
'540/64B',
'236C',
'609/28',
'260/71',
'88/35',
'893',
'149C',
'413/99',
'971/20',
'497/69',
'477/90',
'495A',
'80/48C',
'254B',
'67/56B',
'963/67A',
'194B',
'701/50A',
'200/42C',
'123/75',
'819/61',
'364/42B',
'568/76A',
'844',
'500A',
'381/45A',
'519A',
'94/8C',
'532/29C',
'395/54B',
'979A',
'441/15B',
'295/72C',
'414/15',
'122',
'411/54',
'206/54C',
'946/2',
'895/98A',
'665/41',
'641/61B',
'475/29',
'395C',
'151/85',
'364B',
'141',
'901/45',
'71/21B',
'955/29A',
'40',
'714',
'643B',
'71/2A',
'435B',
'950',
'135A',
'817',
'204/3A',
'512/83A',
'471A',
'997/99',
'10C',
'785/28',
'416C',
'83/69',
'568/83A',
'568/49B',
'66/32C',
'983',
'636/28',
'513',
'160/14',
'785',
'857/38',
'307/69A',
'879C',
'601A',
'201/62A',
'772/61',
'846',
'14B',
'56/27A',
'372',
'136',
'468C',
'656B',
'977/84C',
'834C',
'115/25A',
'215',
'594/24',
'840A',
'726',
'79/40A',
'141A',
'43/46B',
'959C',
'395/37',
'859/22',
'783C',
'451/58',
'36/12',
'731C',
'386',
'272',
'96',
'885C',
'817/99C',
'897',
'321/82C',
'949',
'765/11A',
'119B',
'955',
'962A',
'679/40',
'835',
'449B',
'756C',
'708C',
'84',
'831/92A',
'302C',
'271A',
'502/95',
'632B',
'100/69',
'651B',
'201/64B',
'256',
'865/11A',
'372B',
'892B',
'194/61C',
'236/44A',
'141',
'231B',
'499/66C',
'252B',
'814/39C',
'831/15A',
'437/32',
'553/25',
'186/23A',
'429/39B',
'337',
'567C',
'267/56C',
'217',
'353',
'12A',
'800A',
'279/37',
'230/57',
'124',
'129',
'728/41',
'206/57C',
'697/75C',
'78/1C',
'185',
'12',
'594/18',
'763/71',
'432/41C',
'769/30',
'994/39C',
'470',
'981/93',
'972/99C',
'447',
'747/6A',
'283/75B',
'131',
'811/33',
'980/64A',
'419/18A',
'882/56',
'635',
'57/70B',
'602/5C',
'402',
'29',
'20/7B',
'921/43',
'891B',
'244/43',
'842/82C',
'871/95',
'124/51C',
'885/64',
'907B',
'88',
'629/33B',
'483/83',
'920/68B',
'141B',
'456',
'591/97',
'549/10A',
'459',
'304/13',
'551/65',
'547/32',
'131/93C',
'71/54',
'870/58C',
'413C',
'759',
'15/74B',
'698/21A',
'664/19C',
'282C',
'501/87',
'881/95',
'526/52',
'407',
'409A',
'264/65',
'969/86C',
'425',
'104/15',
'613A',
'400/51',
'997',
'434/85B',
'798A',
'485',
'652',
'16/12',
'601C',
'515',
'898/84',
'355',
'266/11',
'221',
'621',
'934',
'453',
'658/49',
'124/20',
'502/13B',
'393/71B',
'288/97A',
'263/20',
'436',
'945/63',
'796/63C',
'811/21B',
'895/66',
'49',
'719/59',
'164A',
'675/98',
'490A',
'938/26C',
'227',
'183/40',
'606A',
'817/49',
'620B',
'849/40A',
'41C',
'219',
'419',
'610/95',
'443',
'976B',
'896',
'205/93A',
'452/27C',
'783/70A',
'271/53C',
'673/69B',
'584',
'286B',
'337/55C',
'778',
'20/80',
'797A',
'681C',
'16/33B',
'641/86A',
'488',
'70/73B',
'557',
'446/99',
'99A',
'431B',
'755',
'161B',
'566/29',
'898A',
'883',
'241/41',
'594C',
'151',
'353',
'451/21B',
'959/37',
'509/14A',
'8/48',
'224B',
'846/7',
'29/40',
'211/79',
'99/9',
'334B',
'132',
'214/76A',
'806',
'745',
'482B',
'400/15C',
'548/59B',
'308',
'785/84C',
'256/16',
'223B',
'903/95B',
'299/36A',
'807B',
'178/74',
'71B',
'413/39',
'156',
'140/80A',
'12/68C',
'754/5C',
'945B',
'746/88A',
'455',
'674/84C',
'206/9',
'248A',
'428C',
'356/91C',
'38',
'756',
'166C',
'587',
'525',
'261C',
'921A',
'57/24B',
'429/4',
'893/75',
'221/92C',
'426/56A',
'460/49',
'240/33C',
'120',
'668',
'34',
'265/9',
'563',
'377/34',
'176B',
'204/55B',
'393',
'331/52C',
'609/65A',
'737',
'972/83',
'356',
'976/76A',
'822/73A',
'490',
'541B',
'304C',
'993/70C',
'516',
'680/70B',
'6/26',
'186B',
'615',
'981/6',
'951',
'251',
'224/67A',
'462B',
'150/22',
'591B',
'902',
'997',
'633/38',
'316/51',
'901C',
'194B',
'961',
'439/86',
'160C',
'739B',
'489',
'286',
'781/1C',
'177A',
'50/68B',
'927B',
'7/52A',
'736B',
'267B',
'896/98',
'341/80B',
'194C',
'474C',
'554/90C',
'734A',
'161B',
'436/85',
'60A',
'765A',
'493/85B',
'999/32B',
'463',
'225',
'399B',
'615/83',
'362C',
'942/90',
'890A',
'621/23C',
'703',
'518C',
'515A',
'253',
'649B',
'593/97C',
'39',
'970/9C',
'140/50A',
'369',
'198C',
'753',
'57/61A',
'77/52B',
'262A',
'597/82',
'505',
'405/2A',
'226',
'52/27A',
'63',
'751',
'119',
'527',
'591/58B',
'779',
'342/51B',
'158/20',
'906',
'634/6C',
'913/88C',
'160/62C',
'987/98',
'788',
'880',
'449/41B',
'968/72',
'310/59B',
'20/25',
'806/51',
'610/3B',
'197A',
'306',
'868A',
'861/90',
'245C',
'958',
'968/94',
'714',
'843',
'356/28C',
'87/45',
'995',
'797/61',
'617',
'52/5B',
'790B',
'79C',
'243/85C',
'97',
'608',
'323/73B',
'530',
'883/84B',
'369/68',
'899C',
'959/62',
'935',
'961/55B',
'286/55',
'792/29B',
'788',
'91/32',
'709/69A',
'427B',
'120/78',
'909',
'789/82B',
'836',
'684/89',
'357/10A',
'663/68A',
'759',
'22',
'940',
'610',
'118B',
'623',
'700/37C',
'999C',
'681/65B',
'778/35',
'404/77C',
'758A',
'595/12C',
'573',
'817',
'503/15',
'719/15',
'693/34B',
'96/62',
'600C',
'662B',
'47/76',
'80/59A',
'973',
'335/68B',
'23B',
'984',
'727',
'95A',
'73B',
'514A',
'931/53C',
'955/11',
'891B',
'891B',
'140/77B',
'750/40',
'444/30A',
'225B',
'344C',
'615',
'190/17',
'832/82',
'842/31C',
'740',
'305C',
'226C',
'22/62C',
'773C',
'882/9',
'469B',
'928/33',
'572/2C',
'648/79C',
'32/11',
'842/97B',
'143/55A',
'87/30A',
'142/74',
'368',
'708/1',
'728/30B',
'297/39',
'711',
'256/77A',
'949B',
'739/52',
'312A',
'714C',
'441',
'679',
'763A',
'538',
'496B',
'316/91A',
'846',
'255/43B',
'767A',
'947A',
'106/19',
'716/60A',
'940/18',
'816/86A',
'939',
'554A',
'485/88A',
'795A',
'305/16B',
'201/4B',
'508A',
'338',
'147C',
'142',
'518/95C',
'875',
'902',
'179/24B',
'143B',
'589/67',
'820/92A',
'318/94B',
'965',
'80',
'709B',
'258',
'546',
'527/74',
'108C',
'897A',
'247/15C',
'796/81',
'551/54',
'801C',
'17/87B',
'685/15',
'781A',
'184/22',
'980',
'676/93C',
'95',
'904A',
'446',
'177/23',
'916/17',
'416/93A',
'675A',
'661',
'187/61B',
'110/5',
'395',
'763/27',
'128/31',
'33/30C',
'336A',
'928',
'246',
'902C',
'857/37',
'751/71A',
'178/51B',
'691/15B',
'681C',
'265/87A',
'838',
'700',
'962C',
'178/76',
'612/35',
'38A',
'966/46',
'23A',
'109/16A',
'145/46',
'221/17',
'531',
'427/90',
'43',
'194B',
'580/89',
'323',
'720',
'657/79',
'939C',
'52/29',
'100B',
'6',
'492/48C',
'262/21',
'29/90B',
'331',
'718C',
'313',
'614/24',
'289/16',
'997/76A',
'574B',
'453C',
'991/20A',
'627',
'339/56',
'623A',
'913',
'572B',
'718/38',
'194/51',
'742C',
'287/6A',
'619A',
'341/9A',
'746/99B',
'387',
'408/90',
'430A',
'258A',
'205',
'87C',
'792/57',
'207/57B',
'245C',
'820',
'259A',
'639/79A',
'245/31',
'765',
'700',
'631',
'618A',
'26/87',
'584',
'459/55C',
'652A',
'288B',
'527/64A',
'716B',
'158/20',
'784/21',
'296',
'898A',
'266C',
'230/34',
'25C',
'83/9',
'156/93',
'930/10',
'241/8A',
'133/17A',
'665',
'863/78B',
'304A',
'83',
'290A',
'743/10',
'715',
'605/70A',
'481A',
'348/38C',
'854',
'54/96C',
'246/44C',
'990/58B',
'489A',
'899A',
'995/24',
'43B',
'511/20',
'601/29',
'898/61',
'532/99',
'97/70B',
'940/10',
'677',
'301/92A',
'620',
'58',
'169/37',
'820/4',
'760/12A',
'32/69',
'28/76',
'285/99',
'24/86',
'683/16',
'623/90A',
'237/6A',
'967',
'330/29',
'164',
'956/40B',
'713C',
'763/67B',
'602/85B',
'29',
'717A',
'591/38',
'13/40B',
'225',
'765/67A',
'46/59A',
'215/37',
'535',
'766/58',
'973/73',
'838A',
'140/27',
'866',
'660C',
'175/48A',
'410B',
'65B',
'783/80A',
'441/29A',
'734/19',
'362/29C',
'259/33',
'650',
'782/51C',
'738',
'546/10A',
'309C',
'475/90B',
'868C',
'265B',
'426',
'239A',
'982',
'21/75C',
'645/76',
'325/8',
'191B',
'225/37',
'913B',
'788',
'989B',
'865B',
'913/26',
'845/55',
'506B',
'120',
'649B',
'908',
'763A',
'757',
'354A',
'182C',
'643C',
'459',
'388/93B',
'485/46C',
'568/30',
'342/85C',
'677/51A',
'109',
'64/61C',
'829A',
'235',
'693',
'723C',
'939',
'832',
'810/57',
'264/27',
'593',
'217/12',
'342',
'972/57A']
print(li)
li=pd.DataFrame(li)
li.to_csv('list.csv')

['683/50', '19C', '41B', '809A', '364', '695/69', '943/79', '904B', '601', '78', '526C', '252B', '243B', '724A', '700/62', '198B', '533/3C', '492C', '377C', '458/60B', '69/19A', '732/23', '132/77', '975/98B', '451B', '880/36C', '272B', '582C', '952/18', '828/86', '12', '683', '711', '751/18', '663/40', '971/23', '992/3', '112', '724B', '814', '507C', '35/6B', '277/7', '222/8A', '57/72A', '518C', '381', '661', '649/86', '46C', '519', '16A', '113/14B', '238/99B', '394/77A', '320/55B', '721', '286', '729C', '389/62', '626/52B', '550/52', '948A', '819/1B', '82/32', '804/82B', '798/40C', '501/74C', '466/86', '929/81B', '413B', '783B', '216/24', '111', '445/56', '487/94', '925B', '823', '199A', '646/18C', '788', '206', '590/30', '38C', '950', '749/1', '616A', '327', '598/66A', '463/63', '214/59A', '827/54', '36C', '545A', '753/79', '405/53', '782/14B', '558/11C', '486', '658/71', '251/27', '645/9C', '641B', '765/75', '422/97A', '728', '787', '866', '684', '233/85B', '154C', '109/9', '234/51C

In [ ]:
import pandas as pd
import random

# Tên các loại lỗi
error_types = [
    'Lỗi font',
    'Lỗi sai chính tả',
    'Lỗi đổi thứ tự các từ',
    'Lỗi thêm từ gây nhiễu',
    'Lỗi sai chính tả + thêm từ gây nhiễu'
]

# Tạo danh sách các tên đường
street_data = [
    (1, 'Võ Văn Kiệt'), (2, 'Nguyễn Văn Thủ'), (3, 'Hồ Tùng Mậu'),
    (4, 'Phạm Văn Chiêu'), (5, 'Trần Hưng Đạo'), (6, 'Phan Đình Giót'),
    (7, 'Điện Biên Phủ'), (8, 'Trường Chinh'), (9, 'Nguyễn Cư Trinh'),
    (10, 'Nguyễn Văn Luông'), (11, 'Nguyễn Kiệm'), (12, 'Lê Văn Sỹ'),
    (13, 'Phan Xích Long'), (14, 'Đinh Tiên Hoàng'), (15, 'Võ Thị Sáu'),
    (16, 'Lê Hồng Phong'), (17, 'Trần Cao Vân'), (18, 'Bà Huyện Thanh Quan'),
    (19, 'Nguyễn Thị Minh Khai'), (20, 'Trần Quang Khải'), (21, 'Võ Văn Tần'),
    (22, 'Tô Hiến Thành'), (23, 'Nguyễn Văn Trỗi'), (24, 'Dương Bá Trạc'),
    (25, 'Lý Chính Thắng'), (26, 'Đinh Công Tráng'), (27, 'Trần Quốc Toản'),
    (28, 'Nguyễn Thị Định'), (29, 'Lý Tự Trọng'), (30, 'Nguyễn Đình Chiểu'),
    (31, 'Trần Nhật Duật'), (32, 'Hàm Nghi'), (33, 'Nguyễn Thị Nhung'),(34, 'Tân Sơn Nhì'),
    (35, 'Nam Kỳ Khởi Nghĩa'), (36, 'Lê Văn Khương'), (37, 'Phạm Ngọc Thạch'),
    (38, 'Nguyễn Trọng Tuyển'), (39, 'Lạc Long Quân'), (40, 'Lê Duẩn'),
    (41, 'Pasteur'), (42, 'Trần Quốc Thảo'), (43, 'Nguyễn Văn Lượng'),
    (44, 'Hồng Bàng'), (45, 'Hoàng Sa'), (46, 'Nguyễn Thái Học'),
    (47, 'Nguyễn Hữu Cảnh'), (48, 'Đặng Văn Ngữ'), (49, 'Nguyễn Biểu'),
    (50, 'Hoàng Diệu'), (51, 'Xô Viết Nghệ Tĩnh'), (52, 'Nguyễn Đình Chi'),
    (53, 'Chu Mạnh Trinh'), (54, 'Trần Khắc Chân'), (55, 'Phổ Quang'),
    (56, 'Bùi Thị Xuân'), (57, 'Đỗ Quang Đẩu'), (58, 'Ngô Đức Kế'),
    (59, 'Phạm Thế Hiển'), (60, 'Huỳnh Tấn Phát'), (61, 'Trần Bình Trọng'),
    (62, 'Trường Sa'), (63, 'Lê Trọng Tấn'), (64, 'Lê Thánh Tôn'),
    (65, 'Phan Văn Hân'), (66, 'Cao Thắng'), (67, 'Mai Thị Lựu'),
    (68, 'Tôn Đức Thắng'), (69, 'Phạm Văn Hai'), (70, 'Hoàng Hoa Thám'),
    (71, 'Phan Văn Trị'), (72, 'Trần Văn Đang'), (73, 'Nguyễn Oanh'),
    (74, 'Hòa Hảo'), (75, 'Kỳ Đồng'), (76, 'Nguyễn Văn Bảo'),
    (77, 'Dương Quảng Hàm'), (78, 'Lê Quang Định'), (79, 'Lý Phục Man'),
    (80, 'Nguyễn Hồng Đào'), (81, 'Nguyễn Văn Đậu'), (82, 'Âu Cơ'),
    (83, 'Nguyễn Sơn'), (84, 'Nguyễn Hữu Thọ'), (85, 'Trần Thị Nghỉ'),
    (86, 'Nguyễn Hoàng'), (87, 'Phan Đăng Lưu'), (88, 'Cách Mạng Tháng Tám'),
    (89, 'Lũy Bán Bích'), (90, 'Lê Lai'), (91, 'Phan Kế Bính'),
    (92, 'Nguyễn Tri Phương'), (93, 'Hoàng Văn Thụ'), (94, 'Minh Phụng'),
    (95, 'Bình Phú'), (96, 'Thoại Ngọc Hầu'), (97, 'Cộng Hòa'),
    (98, 'Nguyễn Duy'), (99, 'Tạ Quang Bửu'), (100, 'Kha Vạn Cân')
]

# Hàm tạo lỗi font (ví dụ đơn giản là đổi sang chữ hoa hoặc chữ thường)
def error_font(text):
    return text.upper()

# Hàm tạo lỗi sai chính tả (đảo ngẫu nhiên 2 ký tự liên tiếp)
def error_typo(text):
    if len(text) < 2:
        return text
    index = random.randint(0, len(text) - 2)
    return text[:index] + text[index + 1] + text[index] + text[index + 2:]

# Hàm tạo lỗi đổi thứ tự các từ
def error_word_order(text):
    words = text.split()
    if len(words) < 2:
        return text
    random.shuffle(words)
    return ' '.join(words)

# Hàm thêm các từ gây nhiễu
def error_add_noise(text):
    noise_words = ['Ngõ', 'Hẻm', 'Phố', 'Ấp', 'Phường', 'Không biết', 'Không xác định', 'Nhà trống', 'Nhà 2 tầng', 'CSH']
    return text + ' ' + random.choice(noise_words)

# Hàm kết hợp lỗi sai chính tả và thêm từ gây nhiễu
def error_typo_and_noise(text):
    return error_typo(text) + ' ' + random.choice(['phố', 'phường', 'ngõ', 'hẻm'])

# Tạo dataframe ban đầu
data = []

for street_id, street_name in street_data:
    # Thêm 2 hàng không có lỗi
    for _ in range(2):
        data.append([street_id, street_name, random.randint(1, 200), ''])

    # Thêm 5 hàng có lỗi
    data.append([street_id, error_font(street_name), random.randint(1, 200), 'Lỗi font'])
    data.append([street_id, error_typo(street_name), random.randint(1, 200), 'Lỗi sai chính tả'])
    data.append([street_id, error_word_order(street_name), random.randint(1, 200), 'Lỗi đổi thứ tự các từ'])
    data.append([street_id, error_add_noise(street_name), random.randint(1, 200), 'Lỗi thêm từ gây nhiễu'])
    data.append([street_id, error_typo_and_noise(street_name), random.randint(1, 200), 'Lỗi sai chính tả + thêm từ gây nhiễu'])

# Tạo dataframe từ dữ liệu
df = pd.DataFrame(data, columns=['Street_ID', 'Street_Name', 'Số Nhà', 'Loại Lỗi'])

df.to_csv('street_data.csv', index=False, encoding='utf-8-sig')
